In [22]:
import pandas as pd
import os
import json
import argparse

import numpy as np
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

#DATA_DIR = pd.read_csv("input.txt" , sep="\t" )
DATA_DIR = '/content/drive/MyDrive'
MODEL_DIR = 'C:/Users/hp/Downloads/char-rnn-keras-master/model'

BATCH_SIZE = 16
SEQ_LENGTH = 64

In [23]:
def save_weights(epoch, model):
    if not os.path.exists(MODEL_DIR):
        os.makedirs(MODEL_DIR)
    model.save_weights(os.path.join(MODEL_DIR, 'weights.{}.h5'.format(epoch)))

def load_weights(epoch, model):
    model.load_weights(os.path.join(MODEL_DIR, 'weights.{}.h5'.format(epoch)))

In [24]:
def build_model(batch_size, seq_len, vocab_size):
    model = Sequential()
    model.add(Embedding(vocab_size, 512, batch_input_shape=(batch_size, seq_len)))
    for i in range(3):
        model.add(LSTM(256, return_sequences=True, stateful=True))
        model.add(Dropout(0.2))

    model.add(TimeDistributed(Dense(vocab_size))) 
    model.add(Activation('softmax'))
    return model

if __name__ == '__main__':
    model = build_model(16, 64, 50)
    model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (16, 64, 512)             25600     
_________________________________________________________________
lstm_18 (LSTM)               (16, 64, 256)             787456    
_________________________________________________________________
dropout_18 (Dropout)         (16, 64, 256)             0         
_________________________________________________________________
lstm_19 (LSTM)               (16, 64, 256)             525312    
_________________________________________________________________
dropout_19 (Dropout)         (16, 64, 256)             0         
_________________________________________________________________
lstm_20 (LSTM)               (16, 64, 256)             525312    
_________________________________________________________________
dropout_20 (Dropout)         (16, 64, 256)            

In [25]:
def read_batches(T, vocab_size):
    length = T.shape[0]; #129,665
    batch_chars = int(length / BATCH_SIZE); # 8,104

    for start in range(0, batch_chars - SEQ_LENGTH, SEQ_LENGTH): # (0, 8040, 64)
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH)) # 16X64
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, vocab_size)) # 16X64X86
        for batch_idx in range(0, BATCH_SIZE): # (0,16)
            for i in range(0, SEQ_LENGTH): #(0,64)
                X[batch_idx, i] = T[batch_chars * batch_idx + start + i] # 
                Y[batch_idx, i, T[batch_chars * batch_idx + start + i + 1]] = 1
        yield X, Y



In [26]:
class TrainLogger(object):
    def __init__(self, file):
        self.file = os.path.join(LOG_DIR, file)
        self.epochs = 0
        with open(self.file, 'w') as f:
            f.write('epoch,loss,acc\n')

    def add_entry(self, loss, acc):
        self.epochs += 1
        s = '{},{},{}\n'.format(self.epochs, loss, acc)
        with open(self.file, 'a') as f:
            f.write(s)

In [27]:
import sys
sys.argv=['']
del sys


LOG_DIR = 'C:/Users/hp/Downloads/char-rnn-keras-master/logs'

def train(text, epochs=100, save_freq=10):

    # character to index and vice-versa mappings
    char_to_idx = { ch: i for (i, ch) in enumerate(sorted(list(set(text)))) }
    print("Number of unique characters: " + str(len(char_to_idx))) #86

    with open(os.path.join(DATA_DIR, 'char_to_idx.json'), 'w') as f:
        json.dump(char_to_idx, f)

    idx_to_char = { i: ch for (ch, i) in char_to_idx.items() }
    vocab_size = len(char_to_idx)

    #model_architecture
    model = build_model(BATCH_SIZE, SEQ_LENGTH, vocab_size)
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


    #Train data generation
    T = np.asarray([char_to_idx[c] for c in text], dtype=np.int32) #convert complete text into numerical indices

    print("Length of text:" + str(T.size)) #129,665

    steps_per_epoch = (len(text) / BATCH_SIZE - 1) / SEQ_LENGTH  

    log = TrainLogger('training_log.csv')

    for epoch in range(epochs):
        print('\nEpoch {}/{}'.format(epoch + 1, epochs))
        
        losses, accs = [], []

        for i, (X, Y) in enumerate(read_batches(T, vocab_size)):
            
            print(X);

            loss, acc = model.train_on_batch(X, Y)
            print('Batch {}: loss = {}, acc = {}'.format(i + 1, loss, acc))
            losses.append(loss)
            accs.append(acc)

        log.add_entry(np.average(losses), np.average(accs))

        if (epoch + 1) % save_freq == 0:
            save_weights(epoch + 1, model)
            print('Saved checkpoint to', 'weights.{}.h5'.format(epoch + 1))

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Train the model on some text.')
    parser.add_argument('--input', default='input.txt', help='name of the text file to train from')
    parser.add_argument('--epochs', type=int, default=100, help='number of epochs to train for')
    parser.add_argument('--freq', type=int, default=10, help='checkpoint save frequency')
    args = parser.parse_args()

    if not os.path.exists(LOG_DIR):
        os.makedirs(LOG_DIR)

    train(open(os.path.join(DATA_DIR, args.input)).read(), args.epochs, args.freq)


Streaming output truncated to the last 5000 lines.
 [62. 17. 60. ... 29. 60. 84.]
 [62. 17. 61. ... 62.  1. 35.]
 ...
 [84.  3. 32. ... 28. 70.  3.]
 [ 3. 31.  3. ... 58. 77. 58.]
 [60. 28. 84. ... 31. 17. 29.]]
Batch 7: loss = 0.32931947708129883, acc = 0.8818359375
[[28. 84.  3. ...  1.  3. 34.]
 [ 3. 31.  3. ... 29. 60. 84.]
 [78. 71. 77. ... 25. 21. 14.]
 ...
 [28. 18.  1. ... 77. 77. 66.]
 [59. 58. 76. ... 29.  1. 28.]
 [ 1. 60. 17. ...  3. 28. 60.]]
Batch 8: loss = 0.3311230540275574, acc = 0.890625
[[ 3. 34. 18. ...  3. 34.  3.]
 [ 3. 31.  3. ...  1. 61. 17.]
 [23.  0. 38. ... 32. 70.  3.]
 ...
 [71. 64. 65. ... 30.  3. 58.]
 [34. 33. 84. ... 17. 25. 84.]
 [62.  1. 61. ...  1. 40. 78.]]
Batch 9: loss = 0.32104289531707764, acc = 0.87890625
[[64. 17. 64. ...  1. 34. 17.]
 [25. 84.  0. ... 38. 62. 79.]
 [62. 60. 28. ... 62. 61. 60.]
 ...
 [18.  1. 64. ... 33.  3. 58.]
 [ 0. 43. 25. ...  3. 29. 29.]
 [76. 66. 60. ...  3. 61. 17.]]
Batch 10: loss = 0.2923179864883423, acc = 0.897460